In [ ]:
import numpy as np
import obspy
from obspy import UTCDateTime
from ccfj import CC
from ccfj import GetStationPairs
from concurrent.futures import ThreadPoolExecutor
import os
import time
from geopy.distance import great_circle
import folium
import matplotlib.pyplot as plt
import pandas as pd
import h5py

In [ ]:
import sys
sys.path.append(r'../')
from toollib_standard import maplib
from toollib_standard import filelib
from toollib_standard import stacklib

In [ ]:
proj_name = 'output_test_15Hz_zb/'

In [ ]:
filename_basic = proj_name+'Basic_info.npy'
info_basic = np.load(filename_basic, allow_pickle='TRUE').item()      # setting dictionary
dir_stack = info_basic['dir_stack']
dir_stack_all = info_basic['dir_stack_all']
stalistname = info_basic['stalistname']

### Define how many subworks to be done

In [ ]:
key_subworks = ['2010_3']
key_subworks = info_basic['key_subworks']
key_subworks

### Read All data

In [ ]:
filename_subworks = proj_name+'sta_info.npy'
sta_info = np.load(filename_subworks, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
stalist_all = []
with open(stalistname,'r') as f:
    while True:
        tmp = f.readline()
        if tmp:
            stalist_all.append(tmp.split()[0])
            lat_all.append(float(tmp.split()[1]))
            lon_all.append(float(tmp.split()[2]))
        else:
            break
nsta_all = len(stalist_all)
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(nsta_all*(nsta_all-1)/2)

In [ ]:
r = np.zeros(nsta_all)
for i in range(nPairs_all):
    r[i] = great_circle((lat_all[StationPairs_all[i*2]],lon_all[StationPairs_all[i*2]]),(lat_all[StationPairs_all[i*2+1]],lon_all[StationPairs_all[i*2+1]])).km

In [ ]:
stack_file = np.load(dir_stack_all+'summed.npz')
ncfs = stack_file['ncfs']
ncffile = h5py.File('CC_15_zb/ZG_gather.h5','r')
ncfs = ncffile['ncfs'][:]
f = info_basic['f']

stainfo = pd.read_excel('sta.xlsx')
key_pd = 'R0101'
stalist_all = list(stainfo[key_pd])
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)

### Collect Stack files

In [ ]:
for key_subwork in key_subworks:
    print("Collecting ",key_subwork,' ...')
    time0 = time.time()

    nf = info_basic['nf']
    f = info_basic['f']
    dir_stack= info_basic['dir_stack']
    
    nsta = info_basic['nstaS'][key_subwork]
    nPairs = int(nsta*(nsta-1)/2)
    stalistname = proj_name+info_basic['stalistname'] +'-'+key_subwork

    stalist = []
    lon = []
    lat =[]
    with open(stalistname,'r') as f:
        while True:
            tmp = f.readline()
            if tmp:
                stalist.append(tmp.split()[0])
                lat.append(float(tmp.split()[1]))
                lon.append(float(tmp.split()[2]))
            else:
                break

    ncfs_sum_linear = np.zeros((nPairs,nf),dtype=np.complex64)
    r = np.zeros(nPairs)
    count= np.zeros(nPairs)
    StationPairs = GetStationPairs(nsta)
    names = []
    for i in range(nPairs):
        sta1 = StationPairs[2*i]
        sta2 = StationPairs[2*i+1]
        idx1 = int(stalist_all.index(stalist[sta1]))
        idx2 = int(stalist_all.index(stalist[sta2]))
        #idx1 = int(stainfo[stainfo[key_pd]==stalist[sta1]].index.values[0])
        #idx2 = int(stainfo[stainfo[key_pd]==stalist[sta2]].index.values[0])
        
        m = 0
        for j in range(nsta_all-idx1,nsta_all):
            m += j
        num = m + idx2 - idx1 -1
        
        ncfs_sum_linear[i,:] = np.nan_to_num(ncfs[num,:])
        #if count_all[num] > 0:
        #    ncfs_sum_linear[i,:] = ncfs[num,:]/count_all[num]
        #    count[i] = count_all[num]

        r[i] = great_circle((lat[sta1],lon[sta1]),(lat[sta2],lon[sta2])).km
        #names.append([stalist_all[StationPairs_all[2*num]],stalist_all[StationPairs_all[2*num+1]]])
    np.savez(dir_stack+key_subwork+"_summed-linear.npz",ncfs= ncfs_sum_linear,r = r,count = count,stalist=stalist,StationPairs=StationPairs)
    print("Done in ",time.time()-time0," s.")
